# RAG-LLMs

## Setup

### Imports

In [ ]:
# imports
import os
from dotenv import load_dotenv
import pandas as pd
from langchain_openai import ChatOpenAI, AzureChatOpenAI 

from src.utils import *
from src.config import *
from src.data_utils import SQLDBManager, Neo4jGraphManager
from src.tools import GraphQueryAgent, SQLQueryAgent
%load_ext autoreload
%autoreload 2

## SQL RAG

### Prepare & init Postgres DB

In [ ]:
dbm = SQLDBManager.from_env(schema=TABLE_SCHEMA, include_tables=[TABLE_NAME])   # instantiate SQLDBManager
llm = AzureChatOpenAI(model=OPENAI_LLM_VERSION, max_tokens=MAX_TOKENS, temperature=TEMPERATURE)    # instantiate llm

# set db variables
src_table = f'{TABLE_SCHEMA}.{TABLE_NAME}'
data_table = f'{src_table}_filtered'

dbm = SQLDBManager.from_env()  # instantiate SQLDBManager

# filter data table and create new
dbm.filter_table(src_table, REQ_COLS_PATH, TABLE_PRIMARY_KEY, overwrite=True)

# clean html
dbm.clean_html(data_table, [TEXT_COL], TABLE_PRIMARY_KEY) # clean html

# create embeddings
dbm.embed_objs(f'{TABLE_SCHEMA}.{TABLE_NAME}', cols_to_embed=COLS_TO_EMBED, pk=TABLE_PRIMARY_KEY)

### SQL RAG Agent with Tools 

In [ ]:
# use agent w tools to query graph
dqa = SQLQueryAgent(dbm.db, llm)    # instantiate db query agent

# run queries
query = "who in charge of the user story about end-to-end testing of GenAI services?"
print(f"user query: {query}")
result = dqa.run(query)
print(result)

## Graph RAG

### Prepare & init graph DB

In [ ]:
ngm = Neo4jGraphManager.from_env()  # instantiate neo4j graph manager
ngm.from_table(f'{TABLE_SCHEMA}.{TABLE_NAME}', reset=True)    # create graph from table

# embed graph objects
ngm.embed_objs() 

### Graph RAG Agent with tools

In [ ]:
# use agent w tools to query graph
gqa = GraphQueryAgent(ngm.graph, llm)    # instantiate graph query agent

# query = "how many user stories are there in total?"
query = "who in charge of the user story about end-to-end testing of GenAI services?"
print(f"user query: {query}")
gqa.run(query)